<a href="https://colab.research.google.com/github/jvishnuvardhan/Medium_Articles/blob/master/MNIST_tflite_conversion_Quantization_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 322.7MB 46kB/s 
     |████████████████████████████████| 6.5MB 44.3MB/s 
     |████████████████████████████████| 460kB 42.7MB/s 


In [2]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_data=(test_images, test_labels)
)

1875/1875 [==============================] - 21s 11ms/step - loss: 0.2900 - accuracy: 0.9174 - val_loss: 0.1487 - val_accuracy: 0.9571


In [4]:
# Converting to float model (Just conversion)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# Write it out to a tflite file:

tflite_models_dir = pathlib.Path("./mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"mnist_float_model.tflite"
tflite_model_file.write_bytes(tflite_float_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpan37372m/assets


84452

In [5]:
# dynamic range quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmp5d3nquu8/assets


INFO:tensorflow:Assets written to: /tmp/tmp5d3nquu8/assets


23856

In [6]:
# fp16 quantization
converter2 = tf.lite.TFLiteConverter.from_keras_model(model)
converter2.optimizations = [tf.lite.Optimize.DEFAULT]
converter2.target_spec.supported_types = [tf.float16]

tflite_fp16_model = converter2.convert()
tflite_model_fp16_file = tflite_models_dir/"mnist_model_quant_f16.tflite"
tflite_model_fp16_file.write_bytes(tflite_fp16_model)

INFO:tensorflow:Assets written to: /tmp/tmpu7b3vcf_/assets


INFO:tensorflow:Assets written to: /tmp/tmpu7b3vcf_/assets


44272

In [7]:
# Full integer quantization
# This converted model still uses 
converter3 = tf.lite.TFLiteConverter.from_keras_model(model)
converter3.optimizations = [tf.lite.Optimize.DEFAULT]

mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter3.representative_dataset = representative_data_gen

tflite_model_quant = converter3.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_full_int_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmpxgz_efej/assets


INFO:tensorflow:Assets written to: /tmp/tmpxgz_efej/assets


24640

In [8]:
# Full integer quantization
converter4 = tf.lite.TFLiteConverter.from_keras_model(model)
converter4.optimizations = [tf.lite.Optimize.DEFAULT]
converter4.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter4.inference_input_type = tf.uint8
converter4.inference_output_type = tf.uint8

mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter4.representative_dataset = representative_data_gen


tflite_model_quant = converter4.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_full_int_quant_io.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmp9xs64u90/assets


INFO:tensorflow:Assets written to: /tmp/tmp9xs64u90/assets


24720

In [9]:
ls -lh {tflite_models_dir}

total 208K
-rw-r--r-- 1 root root 83K Jul 10 03:31 mnist_float_model.tflite
-rw-r--r-- 1 root root 25K Jul 10 03:31 mnist_model_full_int_quant_io.tflite
-rw-r--r-- 1 root root 25K Jul 10 03:31 mnist_model_full_int_quant.tflite
-rw-r--r-- 1 root root 44K Jul 10 03:31 mnist_model_quant_f16.tflite
-rw-r--r-- 1 root root 24K Jul 10 03:31 mnist_model_quant.tflite


In [10]:
# from google.colab import files

In [11]:
# files.download('/content/mnist_tflite_models/mnist_float_model.tflite')
# files.download('/content/mnist_tflite_models/mnist_model_full_int_quant_io.tflite')
# files.download('/content/mnist_tflite_models/mnist_model_full_int_quant.tflite')
# files.download('/content/mnist_tflite_models/mnist_model_quant_f16.tflite')
# files.download('/content/mnist_tflite_models/mnist_model_quant.tflite')